In [1]:
import os
import json
from datetime import datetime

import pandas as pd

#### Extract

In [2]:
SESSION_FOLDER_PATH = '/home/uchan/Documents/me/browsing-sessions'

FILES = list(map(lambda p: os.path.join(SESSION_FOLDER_PATH, p), os.listdir(SESSION_FOLDER_PATH)))
FILES

['/home/uchan/Documents/me/browsing-sessions/Browsing-Sessions - 2020-08-16 22-22-41.json',
 '/home/uchan/Documents/me/browsing-sessions/884 Sessions - 2022-04-18 06-36-16.json',
 '/home/uchan/Documents/me/browsing-sessions/Sessions - 2020-12-05 15-59-23.json',
 '/home/uchan/Documents/me/browsing-sessions/883 Sessions - 2022-03-25 09-19-12.json',
 '/home/uchan/Documents/me/browsing-sessions/Sessions-Feb2020.json']

In [72]:
def timestamp_to_date(some_date, as_string=True):
    converted_date = datetime.utcfromtimestamp(some_date/1000)
    if as_string:
        return converted_date.strftime('%d-%m-%Y %H:%M:%S')
    return converted_date


def _load_file(file):
    with open(file, 'r') as f:
        data = json.load(f)
    return data


def check_browsing_periods(file):
    print(f'loading {file}')
    data = _load_file(file)
    
    total_record = len(data)
    earliest_record = timestamp_to_date(data[0]['date'])
    latest_record   = timestamp_to_date(data[-1]['date'])
    
    print(f'Total record   : {total_record}')
    print(f'Earliest record: {earliest_record}')
    print(f'Latest record  : {latest_record}')
    print('=' * 15)

In [4]:
for file in FILES:
    check_browsing_periods(file)

loading /home/uchan/Documents/me/browsing-sessions/Browsing-Sessions - 2020-08-16 22-22-41.json
Total record   : 166
Earliest record: 26-10-2019 15:08:59
Latest record  : 23-06-2019 15:19:00
loading /home/uchan/Documents/me/browsing-sessions/884 Sessions - 2022-04-18 06-36-16.json
Total record   : 884
Earliest record: 19-08-2021 00:01:33
Latest record  : 16-01-2022 03:51:30
loading /home/uchan/Documents/me/browsing-sessions/Sessions - 2020-12-05 15-59-23.json
Total record   : 324
Earliest record: 26-10-2019 15:08:59
Latest record  : 13-10-2020 14:46:29
loading /home/uchan/Documents/me/browsing-sessions/883 Sessions - 2022-03-25 09-19-12.json
Total record   : 883
Earliest record: 19-08-2021 00:01:33
Latest record  : 16-01-2022 03:51:30
loading /home/uchan/Documents/me/browsing-sessions/Sessions-Feb2020.json
Total record   : 468
Earliest record: 26-10-2019 15:08:59
Latest record  : 13-10-2020 14:46:29


In [7]:
# FOR EXPLORATION PURPOSES
"""
sample = _load_file(FILES[0])
sample[0].keys()

sample[0]['windows']['1']['2'].keys()
sample[0]['windows']['1']['2']['lastAccessed']
"""

dict_keys(['windows', 'windowsNumber', 'windowsInfo', 'tabsNumber', 'name', 'date', 'tag', 'sessionStartTime', 'id', 'lastEditedTime'])

"""
Data Structure:

Session: 
    ___ attr: windows, tabsNumber, name, date, tag, sessionStartTime, id
Windows:
    ___ attr: id, name, url, lastAccessed

Session: {
    'windows': {
        '1': {
            {
                '1': Windows...,
                '2': Windows...,
                '3': Windows...,
            }
        }
    }
}
"""

In [60]:
def extract_data_from_window(data):
    records = []
    for d in data:
        record = {
                'session_id': d['id'],
                'session_tabsNumber': d['tabsNumber'],
                'session_name': d['name'],
                'session_date': d['date'],
                'session_tag': d['tag'],
                'session_sessionStartTime': d['sessionStartTime'],
        }
        
        for k in d['windows'].values():
            for window in k.values():
                record['window_name'] = window['title']
                record['window_url'] = window['url']
                record['window_lastAccessed'] = window['lastAccessed']
                records.append(record)

    return records

In [61]:
data = []

for file in FILES:
    print(f'processing {file}')
    json_data = _load_file(file)
    data.extend(extract_data_from_window(json_data))
    
len(data)

processing /home/uchan/Documents/me/browsing-sessions/Browsing-Sessions - 2020-08-16 22-22-41.json
processing /home/uchan/Documents/me/browsing-sessions/884 Sessions - 2022-04-18 06-36-16.json
processing /home/uchan/Documents/me/browsing-sessions/Sessions - 2020-12-05 15-59-23.json
processing /home/uchan/Documents/me/browsing-sessions/883 Sessions - 2022-03-25 09-19-12.json
processing /home/uchan/Documents/me/browsing-sessions/Sessions-Feb2020.json


82509

In [62]:
data[0]

{'session_id': '002383d7-cf3d-4d17-b9c2-701bbfb99b22',
 'session_tabsNumber': 6,
 'session_name': 'Networking',
 'session_date': 1572102539130,
 'session_tag': ['networking'],
 'session_sessionStartTime': 1572060943565,
 'window_name': '(1425) Computer Networking: Part 1 of 3 - Georgia Tech - YouTube',
 'window_url': 'https://www.youtube.com/playlist?list=PLAwxTw4SYaPn21MqCiFq2r0FSjk9l6cW2',
 'window_lastAccessed': 1572102336044}

In [63]:
df = pd.DataFrame(data)
df.head()

,session_id,session_tabsNumber,session_name,session_date,session_tag,session_sessionStartTime,window_name,window_url,window_lastAccessed
0,002383d7-cf3d-4d17-b9c2-701bbfb99b22,6,Networking,1572102539130,[networking],1572060943565,(1425) Computer Networking: Part 1 of 3 - Geor...,https://www.youtube.com/playlist?list=PLAwxTw4...,1572102336044
1,002383d7-cf3d-4d17-b9c2-701bbfb99b22,6,Networking,1572102539130,[networking],1572060943565,(1425) Computer Networking: Part 1 of 3 - Geor...,https://www.youtube.com/playlist?list=PLAwxTw4...,1572102336044
2,002383d7-cf3d-4d17-b9c2-701bbfb99b22,6,Networking,1572102539130,[networking],1572060943565,(1425) Computer Networking: Part 1 of 3 - Geor...,https://www.youtube.com/playlist?list=PLAwxTw4...,1572102336044
3,002383d7-cf3d-4d17-b9c2-701bbfb99b22,6,Networking,1572102539130,[networking],1572060943565,(1425) Computer Networking: Part 1 of 3 - Geor...,https://www.youtube.com/playlist?list=PLAwxTw4...,1572102336044
4,002383d7-cf3d-4d17-b9c2-701bbfb99b22,6,Networking,1572102539130,[networking],1572060943565,(1425) Computer Networking: Part 1 of 3 - Geor...,https://www.youtube.com/playlist?list=PLAwxTw4...,1572102336044


In [80]:
CSV_PATH = 'merged_sessions.csv'
df.to_csv(CSV_PATH, index=False)

In [82]:
JSON_PATH = 'merged_sessions.json'
df.to_json(JSON_PATH, indent=4)

#### Transform

In [76]:
# df.session_date = pd.to_datetime(df.session_date, unit='s')
df.session_date = df.session_date.apply(timestamp_to_date, as_string=False)
df.session_sessionStartTime = df.session_sessionStartTime.apply(timestamp_to_date, as_string=False)
df.window_lastAccessed = df.window_lastAccessed.apply(timestamp_to_date, as_string=False)

df.head()

,session_id,session_tabsNumber,session_name,session_date,session_tag,session_sessionStartTime,window_name,window_url,window_lastAccessed
0,002383d7-cf3d-4d17-b9c2-701bbfb99b22,6,Networking,2019-10-26 15:08:59.130,[networking],2019-10-26 03:35:43.565,(1425) Computer Networking: Part 1 of 3 - Geor...,https://www.youtube.com/playlist?list=PLAwxTw4...,2019-10-26 15:05:36.044
1,002383d7-cf3d-4d17-b9c2-701bbfb99b22,6,Networking,2019-10-26 15:08:59.130,[networking],2019-10-26 03:35:43.565,(1425) Computer Networking: Part 1 of 3 - Geor...,https://www.youtube.com/playlist?list=PLAwxTw4...,2019-10-26 15:05:36.044
2,002383d7-cf3d-4d17-b9c2-701bbfb99b22,6,Networking,2019-10-26 15:08:59.130,[networking],2019-10-26 03:35:43.565,(1425) Computer Networking: Part 1 of 3 - Geor...,https://www.youtube.com/playlist?list=PLAwxTw4...,2019-10-26 15:05:36.044
3,002383d7-cf3d-4d17-b9c2-701bbfb99b22,6,Networking,2019-10-26 15:08:59.130,[networking],2019-10-26 03:35:43.565,(1425) Computer Networking: Part 1 of 3 - Geor...,https://www.youtube.com/playlist?list=PLAwxTw4...,2019-10-26 15:05:36.044
4,002383d7-cf3d-4d17-b9c2-701bbfb99b22,6,Networking,2019-10-26 15:08:59.130,[networking],2019-10-26 03:35:43.565,(1425) Computer Networking: Part 1 of 3 - Geor...,https://www.youtube.com/playlist?list=PLAwxTw4...,2019-10-26 15:05:36.044


#### Load (Later Analysis)

In [100]:
years_and_months = set([
    (year, month) for year in range(2018,2023) for month in range(1,13)
])

In [101]:
def extract_years_and_months(date):
    return (date.year, date.month)

In [102]:
data_years_and_months = df.session_date.apply(extract_years_and_months)
data_years_and_months = set(data_years_and_months)

In [103]:
# it's dates that are not on the data.
# possible explanation:
#     2019-1 ~ 2019-3 I was on an internship;
#     2019-7 ~ 2019-8 I was on a social service
years_and_months.difference(data_years_and_months)

{(2018, 1),
 (2018, 2),
 (2018, 3),
 (2018, 4),
 (2018, 5),
 (2018, 6),
 (2018, 7),
 (2018, 8),
 (2018, 9),
 (2018, 10),
 (2018, 11),
 (2019, 1),
 (2019, 2),
 (2019, 3),
 (2019, 7),
 (2019, 8),
 (2022, 5),
 (2022, 6),
 (2022, 7),
 (2022, 8),
 (2022, 9),
 (2022, 10),
 (2022, 11),
 (2022, 12)}